# Solution To The Problem Statement 

In [ ]:
#importing the libraries

import urllib
from bs4 import BeautifulSoup, Tag
import pandas as pd

In [ ]:
url = "http://www.fallingrain.com" 
page = urllib.request.urlopen(url+"/world/IN/")
soup = BeautifulSoup(page, 'html.parser')           # Extracting the home page into a soup object

In [ ]:
df = pd.DataFrame(columns=['State','City','Latitude','Longitude','Elevation','Estimated Population'] )    # Creating a DataFrame

In [ ]:
for state in soup.find("ul").contents:   # Since the name of the states were listed under the <ul> tag
    if isinstance(state, Tag):
        name = state.text
        state_soup = BeautifulSoup(urllib.request.urlopen(url+state.a["href"]), 'html.parser') # Creating a soup object of the link
        df = extract_data(state_soup, name, df)   # Function Call
            

In [ ]:
df

In [ ]:
df.to_csv("Directory_of_Regions_In_India.csv", index =False, encoding='utf-8') # Converting to CSV

In [ ]:
# THE EXTRACT_DATA FUNCTION

def extract_data(soup, name,df):
    cur = soup.find(string="Alphabetical listing of Places in "+name) # Bringing the current location to the heading
    if cur.find_next(string="Name"):  # To check if its the required table under it or not
        for var in cur.find_next("table").contents: 
            if isinstance(var, Tag):         
                if var.find(string="Name"):   # To skip the first row of column names
                    continue
                row = var.find_all(string=True)   # Making a list of the row 
                if(name=="None"):
                    row.insert(4, "None")    # Since None doesn't have a string under the column Region
                print(row)                   # Just to check the correct Values while execution
                   
                # Appending the required values to the dataframe 
                df = df.append({'State':row[4],'City':row[0],'Latitude':row[8],'Longitude':row[10],'Elevation':row[12],'Estimated Population':row[14]}, ignore_index=True)
                                               
    else:
        for var in cur.find_all_next("a"):
            print(var)      # To check the link being extracted
            soup = BeautifulSoup(urllib.request.urlopen(url+var["href"]), 'html.parser') 
            df =  extract_data(soup, name, df)   # Extracting each link and recursing untill table is found
  
            
    return df    